In [ ]:
############################################################
# 1) IMPORTS
############################################################
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, mannwhitneyu
from statsmodels.stats.multitest import multipletests

############################################################
# 2) CUSTOM EFFECT SIZE FUNCTIONS
############################################################

def cohen_d(x, y):
    """
    Computes Cohen's d for two independent samples x, y.
    Formula:
        d = (mean_x - mean_y) / pooled_std
    where
        pooled_std = sqrt( ((n_x - 1)*var_x + (n_y - 1)*var_y ) / (n_x + n_y - 2) ).
    """
    x = np.array(x, dtype=float)
    y = np.array(y, dtype=float)
    x = x[~np.isnan(x)]
    y = y[~np.isnan(y)]

    n_x, n_y = len(x), len(y)
    if n_x < 2 or n_y < 2:
        return np.nan  # Not enough data

    mean_x, mean_y = np.mean(x), np.mean(y)
    var_x, var_y = np.var(x, ddof=1), np.var(y, ddof=1)

    # Pooled standard deviation
    pooled_var = ((n_x - 1) * var_x + (n_y - 1) * var_y) / (n_x + n_y - 2)
    pooled_std = np.sqrt(pooled_var)
    if pooled_std == 0:
        return np.nan  # Avoid divide-by-zero

    return (mean_x - mean_y) / pooled_std


def cliffs_delta(x, y):
    """
    Computes Cliff's Delta for two samples x, y.
    Cliff's Delta = (# of x>y pairs - # of x<y pairs) / (n_x * n_y).
    Returns float in [-1, +1].
    """
    x = np.array(x, dtype=float)
    y = np.array(y, dtype=float)
    x = x[~np.isnan(x)]
    y = y[~np.isnan(y)]

    n_x, n_y = len(x), len(y)
    if n_x == 0 or n_y == 0:
        return np.nan

    bigger = 0
    smaller = 0
    for val_x in x:
        bigger += np.sum(val_x > y)
        smaller += np.sum(val_x < y)

    return (bigger - smaller) / float(n_x * n_y)

############################################################
# 3) STATISTICAL TEST HELPERS
############################################################

def compare_periods_parametric(series_new, series_old):
    """
    Returns p-value using Welch's t-test (independent, unequal variances).
    """
    t_stat, p_val = ttest_ind(series_new, series_old, equal_var=False, nan_policy='omit')
    return p_val

def compare_periods_nonparametric(series_new, series_old):
    """
    Returns p-value for Mann-Whitney U test (two-sided).
    """
    u_stat, p_val = mannwhitneyu(series_new, series_old, alternative='two-sided')
    return p_val

############################################################
# 4) CORE ANALYSIS FUNCTIONS
############################################################

def run_grouping_analysis(df_new, df_old, grouping, grouping_value, threshold):
    """
    Performs analysis for a specific grouping (Overall, Category, Shortname).
    Returns a dictionary with all required metrics.
    """
    if grouping == 'Overall':
        scores_new = df_new['score']
        scores_old = df_old['score']
    else:
        scores_new = df_new[df_new[grouping.lower()] == grouping_value]['score']
        scores_old = df_old[df_old[grouping.lower()] == grouping_value]['score']

    # Ensure there are enough data points
    if len(scores_new) < 2 or len(scores_old) < 2:
        return None  # Not enough data to perform tests

    # Calculate p-values
    param_p = compare_periods_parametric(scores_new, scores_old)
    nonparam_p = compare_periods_nonparametric(scores_new, scores_old)

    # Calculate effect sizes
    cohens_d_val = cohen_d(scores_new, scores_old)
    cliffs_delta_val = cliffs_delta(scores_new, scores_old)

    # Calculate absolute percentage difference based on old scores
    mean_new = np.mean(scores_new)
    mean_old = np.mean(scores_old)
    if mean_old == 0:
        pct_diff = np.nan  # Avoid division by zero
    else:
        pct_diff = abs((mean_new - mean_old) / mean_old) * 100

    # Determine if percentage difference meets/exceeds threshold
    if pd.isna(pct_diff):
        pct_significant = False
    else:
        pct_significant = pct_diff >= threshold

    # Compile results
    result = {
        'Assessment Type': df_new['assessmenttype'].iloc[0],
        'Grouping': grouping,
        'Grouping Value': grouping_value,
        'Comparison': 'EOY2024_vs_MY2024',
        'Parametric P-Value (Raw)': param_p,
        'Non-Parametric P-Value (Raw)': nonparam_p,
        'Cohen\'s d': cohens_d_val,
        'Cliff\'s Delta': cliffs_delta_val,
        'Absolute % Difference': pct_diff,
        'Threshold (%)': threshold,
        'Significant (Threshold)': pct_significant
    }

    return result

############################################################
# 5) LOAD AND PREPARE DATA
############################################################

# Define the path to your Excel file
file_path = r"C:\Users\DavidShevchenko\Downloads"

df = pd.read_excel(file_path, sheet_name='here')

df_filtered = df[
    (df['assessmenttype'].isin(['1', '2'])) &
    (df['period'].isin(['x', 'y']))
].copy()

############################################################
# 6) DEFINE GROUPINGS
############################################################

# Define the grouping levels
grouping_levels = ['Overall', 'Category', 'Detailed']

# Get unique categories and shortnames from the filtered data
unique_categories = df_filtered['category'].dropna().unique()
unique_shortnames = df_filtered['Detailed'].dropna().unique()

############################################################
# 7) DETERMINING DYNAMIC PERCENTAGE DIFFERENCE THRESHOLDS
############################################################

def determine_thresholds(df, grouping_levels, percentile=75):
    thresholds = {}
    for grouping in grouping_levels:
        if grouping == 'Overall':
            # Fixed threshold for Overall
            thresholds[grouping] = 5  # Example: 5%
        else:
            # Calculate percentage differences for each unique value in the grouping
            pct_diffs = []
            if grouping == 'Category':
                unique_values = unique_categories
            else:
                unique_values = unique_shortnames

            for value in unique_values:
                subset_new = df[
                    (df['period'] == 'EOY 2024') &
                    (df[grouping.lower()] == value)
                ]['score']
                subset_old = df[
                    (df['period'] == 'MY 2024') &
                    (df[grouping.lower()] == value)
                ]['score']

                if len(subset_new) < 1 or len(subset_old) < 1:
                    continue  # Skip if no data

                mean_new = subset_new.mean()
                mean_old = subset_old.mean()

                if mean_old == 0:
                    continue  # Avoid division by zero

                pct_diff = abs((mean_new - mean_old) / mean_old) * 100
                pct_diffs.append(pct_diff)

            if len(pct_diffs) == 0:
                thresholds[grouping] = 0  # Default threshold if no data
            else:
                # Set threshold at the specified percentile
                thresholds[grouping] = np.percentile(pct_diffs, percentile)

    return thresholds

# Determine dynamic thresholds
dynamic_thresholds = determine_thresholds(df_filtered, grouping_levels, percentile=75)

############################################################
# 8) PERFORM ANALYSIS AND COMPILE RESULTS
############################################################

# Initialize a list to store all results
results = []

# Loop through each Assessment Type
assessment_types = df_filtered['assessmenttype'].dropna().unique()

for assessment in assessment_types:
    df_assessment_new = df_filtered[
        (df_filtered['assessmenttype'] == assessment) &
        (df_filtered['period'] == 'EOY 2024')
    ]
    df_assessment_old = df_filtered[
        (df_filtered['assessmenttype'] == assessment) &
        (df_filtered['period'] == 'MY 2024')
    ]

    for grouping in grouping_levels:
        if grouping == 'Overall':
            threshold = dynamic_thresholds[grouping]
            result = run_grouping_analysis(
                df_new=df_assessment_new,
                df_old=df_assessment_old,
                grouping=grouping,
                grouping_value='Overall',
                threshold=threshold
            )
            if result:
                results.append(result)
        else:
            if grouping == 'Category':
                unique_values = unique_categories
            else:
                unique_values = unique_shortnames
            for value in unique_values:
                threshold = dynamic_thresholds[grouping]
                result = run_grouping_analysis(
                    df_new=df_assessment_new[
                        df_assessment_new[grouping.lower()] == value
                    ],
                    df_old=df_assessment_old[
                        df_assessment_old[grouping.lower()] == value
                    ],
                    grouping=grouping,
                    grouping_value=value,
                    threshold=threshold
                )
                if result:
                    results.append(result)

# Convert the results list to a DataFrame
reporting_table = pd.DataFrame(results)

############################################################
# 9) APPLY BONFERRONI CORRECTION TO P-VALUES
############################################################

# Determine the number of tests performed
num_tests = len(reporting_table)

# Apply Bonferroni correction to parametric p-values
reporting_table['Parametric P-Value Corrected'] = reporting_table['Parametric P-Value (Raw)'] * num_tests
reporting_table['Parametric P-Value Corrected'] = reporting_table['Parametric P-Value Corrected'].apply(lambda x: min(x, 1))

# Apply Bonferroni correction to non-parametric p-values
reporting_table['Non-Parametric P-Value Corrected'] = reporting_table['Non-Parametric P-Value (Raw)'] * num_tests
reporting_table['Non-Parametric P-Value Corrected'] = reporting_table['Non-Parametric P-Value Corrected'].apply(lambda x: min(x, 1))

# Determine significance flags based on corrected p-values
reporting_table['Parametric Significant'] = reporting_table['Parametric P-Value Corrected'] < 0.05
reporting_table['Non-Parametric Significant'] = reporting_table['Non-Parametric P-Value Corrected'] < 0.05

# Determine combined significance flag (both tests are significant)
reporting_table['Combined Significant'] = reporting_table['Parametric Significant'] & reporting_table['Non-Parametric Significant']

############################################################
# 10) FINAL REPORTING TABLE STRUCTURE
############################################################

# Rearrange columns to include corrected p-values and significance flags
reporting_table = reporting_table[[
    'Assessment Type',
    'Grouping',
    'Grouping Value',
    'Comparison',
    'Parametric P-Value (Raw)',
    'Non-Parametric P-Value (Raw)',
    'Parametric P-Value Corrected',
    'Non-Parametric P-Value Corrected',
    'Parametric Significant',
    'Non-Parametric Significant',
    'Combined Significant',
    'Cohen\'s d',
    'Cliff\'s Delta',
    'Absolute % Difference',
    'Threshold (%)',
    'Significant (Threshold)'
]]

############################################################
# 11) FORMAT P-VALUES FOR CLARITY
############################################################

# Define a function to format p-values in scientific notation with 2 decimal places
def format_pval(x):
    if pd.isna(x):
        return ''
    return "{0:.2e}".format(x)

# Apply formatting
reporting_table['Parametric P-Value (Raw)'] = reporting_table['Parametric P-Value (Raw)'].apply(format_pval)
reporting_table['Non-Parametric P-Value (Raw)'] = reporting_table['Non-Parametric P-Value (Raw)'].apply(format_pval)
reporting_table['Parametric P-Value Corrected'] = reporting_table['Parametric P-Value Corrected'].apply(format_pval)
reporting_table['Non-Parametric P-Value Corrected'] = reporting_table['Non-Parametric P-Value Corrected'].apply(format_pval)

############################################################
# 12) EXPORT THE REPORTING TABLE TO EXCEL
############################################################

# Define the output Excel file path
output_file = r"C:\Users\DavidShevchenko\Downloads"

# Export to Excel
reporting_table.to_excel(output_file, index=False, sheet_name='Reporting Table')

# Inform the user
print(f"Detailed reporting table has been exported to '{output_file}'.")
